In [12]:
# https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes
# https://scikit-learn.org/stable/modules/tree.html#classification
# https://scikit-learn.org/stable/modules/sgd.html#classification
# https://scikit-learn.org/stable/modules/svm.html#classification
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html#sklearn.ensemble.ExtraTreesClassifier

In [35]:
import math
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn import preprocessing

import numpy as np
import pandas as pd

import time
import random
import matplotlib.pyplot as plt
from scipy import interp
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from collections import Counter
from tqdm import tqdm

In [36]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report

In [56]:
def load_data(directory):
    #D_SSM = np.loadtxt(directory + '/D_SM.txt')
    D_SSM = np.loadtxt(directory + '/D_SM.txt')
    M_FSM = np.loadtxt(directory + '/M_SM.txt')
    print("D_SSM",D_SSM.shape)
    print("M_FSM",M_FSM.shape)

    #D_SSM = np.loadtxt(directory + '/D_SM.txt')



    ID = np.zeros(shape=(D_SSM.shape[0], D_SSM.shape[1]))
    IM = np.zeros(shape=(M_FSM.shape[0], M_FSM.shape[1]))

    for i in range(D_SSM.shape[0]):
        for j in range(D_SSM.shape[1]):
            if D_SSM[i][j] == 0:
                ID[i][j] = D_SSM[i][j]######D_GSM[i][j]
            else:
                ID[i][j] = D_SSM[i][j]
    for i in range(M_FSM.shape[0]):
        for j in range(M_FSM.shape[1]):
            if M_FSM[i][j] == 0:
                IM[i][j] = M_FSM[i][j]#######M_GSM[i][j]
            else:
                IM[i][j] = M_FSM[i][j]
                
    ID = pd.DataFrame(ID).reset_index()
    IM = pd.DataFrame(IM).reset_index()
    ID.rename(columns = {'index':'id'}, inplace = True)
    IM.rename(columns = {'index':'id'}, inplace = True)
    ID['id'] = ID['id'] + 1
    IM['id'] = IM['id'] + 1
    
    return ID, IM

def sample(directory, random_seed):
    all_associations = pd.read_csv(directory + '/drug_mutation_pairs.csv', names=['Drug', 'Mutation', 'label'])
    known_associations = all_associations.loc[all_associations['label'] == 1]
    known_associations_resistance=all_associations.loc[all_associations['label'] == -1]
    unknown_associations = all_associations.loc[all_associations['label'] == 0]
    #random_negative = unknown_associations.sample(n=known_associations.shape[0], random_state=random_seed, axis=0)
    random_negative = unknown_associations.sample(n=known_associations.shape[0]+known_associations_resistance.shape[0], random_state=random_seed, axis=0)

    sample_df = known_associations.append(random_negative)
    sample_df.reset_index(drop=True, inplace=True)

    return sample_df

In [57]:
def performances(y_true, y_pred, y_prob):

    print('y_true:',y_true)
    print('y_pred:',y_pred)
    #tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel().tolist()

    pos_acc = tp / sum(y_true)
    neg_acc = tn / (len(y_pred) - sum(y_pred)) # [y_true=0 & y_pred=0] / y_pred=0
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    recall = tp / (tp+fn)
    precision = tp / (tp+fp)
    f1 = 2*precision*recall / (precision+recall)
    
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
    print('y_pred: -1 = {} |0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1],Counter(y_pred)[2]))###
    print('y_true: -1 = {} | 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1], Counter(y_true)[2]))###
    print('acc={:.4f}|precision={:.4f}|recall={:.4f}|f1={:.4f}|auc={:.4f}|aupr={:.4f}|pos_acc={:.4f}|neg_acc={:.4f}'.format(accuracy, precision, recall, f1, roc_auc, aupr, pos_acc, neg_acc))
    return (y_true, y_pred, y_prob), (accuracy, precision, recall, f1, roc_auc, aupr, pos_acc, neg_acc)

In [58]:
def obtain_data(directory, isbalance):

    ID, IM = load_data(directory)

    if isbalance:
        dtp = sample(directory, random_seed = 1234)
    else:
        dtp = pd.read_csv(directory + '/drug_mutation_pairs.csv', names=['Drug', 'Mutation', 'label'])

    mirna_ids = list(set(dtp['Drug']))
    disease_ids = list(set(dtp['Mutation']))
    random.shuffle(mirna_ids)
    random.shuffle(disease_ids)
    print('# Drug = {} | Mutation = {}'.format(len(mirna_ids), len(disease_ids)))

    mirna_test_num = int(len(mirna_ids) / 5)
    disease_test_num = int(len(disease_ids) / 5)
    print('# Test: Drug = {} | Mutation = {}'.format(mirna_test_num, disease_test_num))  
    print(dtp)
    print("ID",ID)
    print("IM",IM)
    print(dtp.shape)
    print("ID",ID.shape)
    print("IM",IM.shape)    
    
    samples = pd.merge(pd.merge(dtp, ID, left_on = 'Drug', right_on = 'id'), IM, left_on = 'Mutation', right_on = 'id')
    samples.drop(labels = ['id_x', 'id_y'], axis = 1, inplace = True)
    
    return ID, IM, dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples

In [59]:
def generate_task_Tp_train_test_idx(samples):
    kf = KFold(n_splits = 5, shuffle = True, random_state = 1234)

    train_index_all, test_index_all, n = [], [], 0
    train_id_all, test_id_all = [], []
    fold = 0
    for train_idx, test_idx in tqdm(kf.split(samples.iloc[:, 3:])): #train_index与test_index为下标
        print('-------Fold ', fold)
        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)

        train_id_all.append(np.array(dtp.iloc[train_idx][['Drug', 'Mutation']]))
        test_id_all.append(np.array(dtp.iloc[test_idx][['Drug', 'Mutation']]))

        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        fold += 1
    return train_index_all, test_index_all, train_id_all, test_id_all

In [60]:
def generate_task_Tm_Td_train_test_idx(item, ids, dtp):
    
    test_num = int(len(ids) / 5)
    
    train_index_all, test_index_all = [], []
    train_id_all, test_id_all = [], []
    
    for fold in range(5):
        print('-------Fold ', fold)
        if fold != 4:
            test_ids = ids[fold * test_num : (fold + 1) * test_num]
        else:
            test_ids = ids[fold * test_num :]

        train_ids = list(set(ids) ^ set(test_ids))
        print('# {}: Train = {} | Test = {}'.format(item, len(train_ids), len(test_ids)))

        test_idx = dtp[dtp[item].isin(test_ids)].index.tolist()
        train_idx = dtp[dtp[item].isin(train_ids)].index.tolist()
        random.shuffle(test_idx)
        random.shuffle(train_idx)
        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        assert len(train_idx) + len(test_idx) == len(dtp)

        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)
        
        train_id_all.append(train_ids)
        test_id_all.append(test_ids)
        
    return train_index_all, test_index_all, train_id_all, test_id_all

In [61]:
def run_rf(train_index_all, test_index_all, samples, classfier):
    
    fold = 0
    for train_idx, test_idx in zip(train_index_all, test_index_all):
        print('-----------------------Fold = ', str(fold))

        X = samples.iloc[:, 3:]
        y = samples['label']

        scaler = preprocessing.MinMaxScaler().fit(X.iloc[train_idx,:])
        X = scaler.transform(X)

        x_train, y_train = X[train_idx], y[train_idx]
        x_test, y_test = X[test_idx], y[test_idx]

        if classfier == 'ERT':
            clf = ExtraTreesClassifier(random_state = 19961231)
        elif classfier == 'GNB':
            clf = GaussianNB()
        elif classfier == 'DT':
            clf = DecisionTreeClassifier(random_state = 19961231)
        elif classfier == 'SGD':
            clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
        elif classfier == 'SVM':
            clf = svm.SVC()
            
        clf.fit(x_train, y_train)

        y_train_prob = clf.predict_proba(x_train)
        y_test_prob = clf.predict_proba(x_test)

        y_train_pred = clf.predict(x_train)
        y_test_pred = clf.predict(x_test)

        print('Train:')
        ys_train, metrics_train = performances(y_train, y_train_pred, y_train_prob[:, 1])
        print('Test:')
        ys_test, metrics_test = performances(y_test, y_test_pred, y_test_prob[:, 1])

        fold += 1
    
    return ys_train, metrics_train, ys_test, metrics_test

In [62]:
directory='C:/Users/xs/Desktop/图采样data/last data/'
#directory = 'data'
for isbalance in [True]:
#for isbalance in [True, False]:
    
    ID, IM, dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples = obtain_data(directory, 
                                                                                                 isbalance)
    for task in ['Tp', 'Tm', 'Td']:
        
        print('========== isbalance = {} | task = {}'.format(isbalance, task))
        
        if task == 'Tp':
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(samples)
            
        elif task == 'Tm':
            item = 'Drug'
            ids = mirna_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        elif task == 'Td':
            item = 'Mutation'
            ids = disease_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        ys_train, metrics_train, ys_test, metrics_test = run_rf(train_index_all, test_index_all, samples, 'ERT')

D_SSM (184, 184)
M_FSM (661, 661)


5it [00:00, 385.65it/s]

# Drug = 184 | Mutation = 640
# Test: Drug = 36 | Mutation = 128
      Drug  Mutation  label
0        1       154      1
1        2       528      1
2        3       451      1
3        5       277      1
4        5       564      1
...    ...       ...    ...
2584     8       252      0
2585    32        31      0
2586   180       656      0
2587    20        81      0
2588   101       191      0

[2589 rows x 3 columns]
ID       id         0         1         2         3         4         5  \
0      1  1.000000  0.897662  0.969539  0.977718  0.895714  0.955645   
1      2  0.897662  1.000000  0.973867  0.967525  0.998237  0.974889   
2      3  0.969539  0.973867  1.000000  0.997122  0.973182  0.986712   
3      4  0.977718  0.967525  0.997122  1.000000  0.964158  0.991381   
4      5  0.895714  0.998237  0.973182  0.964158  1.000000  0.966592   
..   ...       ...       ...       ...       ...       ...       ...   
179  180  0.862918  0.675884  0.775158  0.802688  0.655391  0.79630

KeyboardInterrupt: 

In [56]:
directory='C:/Users/Administrator/Desktop/图采样data/last data'
#directory = 'data'
for isbalance in [True]:
#for isbalance in [True, False]:
    
    ID, IM, dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples = obtain_data(directory, 
                                                                                                 isbalance)
    for task in ['Tp', 'Tm', 'Td']:
        
        print('========== isbalance = {} | task = {}'.format(isbalance, task))
        
        if task == 'Tp':
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(samples)
            
        elif task == 'Tm':
            item = 'Drug'
            ids = mirna_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        elif task == 'Td':
            item = 'Mutation'
            ids = disease_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        ys_train, metrics_train, ys_test, metrics_test = run_rf(train_index_all, test_index_all, samples, 'GNB')

5it [00:00, 501.36it/s]

# Drug = 183 | Mutation = 599
# Test: Drug = 36 | Mutation = 119
========== isbalance = True | task = Tp
-------Fold  0
# Pairs: Train = 1336 | Test = 334
-------Fold  1
# Pairs: Train = 1336 | Test = 334
-------Fold  2
# Pairs: Train = 1336 | Test = 334
-------Fold  3
# Pairs: Train = 1336 | Test = 334
-------Fold  4
# Pairs: Train = 1336 | Test = 334
-----------------------Fold =  0
Train:
tn = 382, fp = 294, fn = 303, tp = 357
y_pred: 0 = 685 | 1 = 651
y_true: 0 = 676 | 1 = 660
acc=0.5531|precision=0.5484|recall=0.5409|f1=0.5446|auc=0.5679|aupr=0.6238|pos_acc=0.5409|neg_acc=0.5577
Test:
tn = 96, fp = 63, fn = 83, tp = 92
y_pred: 0 = 179 | 1 = 155
y_true: 0 = 159 | 1 = 175
acc=0.5629|precision=0.5935|recall=0.5257|f1=0.5576|auc=0.5718|aupr=0.6443|pos_acc=0.5257|neg_acc=0.5363
-----------------------Fold =  1


Train:
tn = 320, fp = 338, fn = 282, tp = 396
y_pred: 0 = 602 | 1 = 734
y_true: 0 = 658 | 1 = 678
acc=0.5359|precision=0.5395|recall=0.5841|f1=0.5609|auc=0.5500|aupr=0.6398|pos_acc=0.5841|neg_acc=0.5316
Test:
tn = 82, fp = 95, fn = 61, tp = 96
y_pred: 0 = 143 | 1 = 191
y_true: 0 = 177 | 1 = 157
acc=0.5329|precision=0.5026|recall=0.6115|f1=0.5517|auc=0.5652|aupr=0.6287|pos_acc=0.6115|neg_acc=0.5734
-----------------------Fold =  2
Train:
tn = 332, fp = 333, fn = 273, tp = 398
y_pred: 0 = 605 | 1 = 731
y_true: 0 = 665 | 1 = 671
acc=0.5464|precision=0.5445|recall=0.5931|f1=0.5678|auc=0.5608|aupr=0.6499|pos_acc=0.5931|neg_acc=0.5488
Test:
tn = 71, fp = 99, fn = 66, tp = 98
y_pred: 0 = 137 | 1 = 197
y_true: 0 = 170 | 1 = 164
acc=0.5060|precision=0.4975|recall=0.5976|f1=0.5429|auc=0.5188|aupr=0.6221|pos_acc=0.5976|neg_acc=0.5182
-----------------------Fold =  3
Train:
tn = 344, fp = 329, fn = 268, tp = 395
y_pred: 0 = 612 | 1 = 724
y_true: 0 = 673 | 1 = 663
acc=0.5531|precision=0.5456|recall

In [66]:
directory='C:/Users/Administrator/Desktop/图采样data/last data'
#directory = 'data'
for isbalance in [True]:
#for isbalance in [True, False]:
    
    ID, IM, dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples = obtain_data(directory, 
                                                                                                 isbalance)
    for task in ['Tp', 'Tm', 'Td']:
        
        print('========== isbalance = {} | task = {}'.format(isbalance, task))
        
        if task == 'Tp':
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(samples)
            
        elif task == 'Tm':
            item = 'Drug'
            ids = mirna_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        elif task == 'Td':
            item = 'Mutation'
            ids = disease_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)
            
        ys_train, metrics_train, ys_test, metrics_test = run_rf(train_index_all, test_index_all, samples, 'DT')

5it [00:00, 501.32it/s]

# Drug = 184 | Mutation = 603
# Test: Drug = 36 | Mutation = 120
========== isbalance = True | task = Tp
-------Fold  0
# Pairs: Train = 1470 | Test = 368
-------Fold  1
# Pairs: Train = 1470 | Test = 368
-------Fold  2
# Pairs: Train = 1470 | Test = 368
-------Fold  3
# Pairs: Train = 1471 | Test = 367
-------Fold  4
# Pairs: Train = 1471 | Test = 367
-----------------------Fold =  0


Train:
tn = 750, fp = 0, fn = 0, tp = 720
y_pred: 0 = 750 | 1 = 720
y_true: 0 = 750 | 1 = 720
acc=1.0000|precision=1.0000|recall=1.0000|f1=1.0000|auc=1.0000|aupr=1.0000|pos_acc=1.0000|neg_acc=1.0000
Test:
tn = 119, fp = 50, fn = 51, tp = 148
y_pred: 0 = 170 | 1 = 198
y_true: 0 = 169 | 1 = 199
acc=0.7255|precision=0.7475|recall=0.7437|f1=0.7456|auc=0.7239|aupr=0.8149|pos_acc=0.7437|neg_acc=0.7000
-----------------------Fold =  1
Train:
tn = 726, fp = 0, fn = 0, tp = 744
y_pred: 0 = 726 | 1 = 744
y_true: 0 = 726 | 1 = 744
acc=1.0000|precision=1.0000|recall=1.0000|f1=1.0000|auc=1.0000|aupr=1.0000|pos_acc=1.0000|neg_acc=1.0000
Test:
tn = 139, fp = 54, fn = 36, tp = 139
y_pred: 0 = 175 | 1 = 193
y_true: 0 = 193 | 1 = 175
acc=0.7554|precision=0.7202|recall=0.7943|f1=0.7554|auc=0.7572|aupr=0.8062|pos_acc=0.7943|neg_acc=0.7943
-----------------------Fold =  2
Train:
tn = 738, fp = 0, fn = 0, tp = 732
y_pred: 0 = 738 | 1 = 732
y_true: 0 = 738 | 1 = 732
acc=1.0000|precision=1.0000|recall=1.0000|

In [67]:
directory='C:/Users/Administrator/Desktop/图采样data/last data'
#directory = 'data'
for isbalance in [True]:
#for isbalance in [True, False]:
    
    ID, IM, dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples = obtain_data(directory, 
                                                                                                 isbalance)
    for task in ['Tp', 'Tm', 'Td']:
        
        print('========== isbalance = {} | task = {}'.format(isbalance, task))
        
        if task == 'Tp':
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(samples)
            
        elif task == 'Tm':
            item = 'Drug'
            ids = mirna_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        elif task == 'Td':
            item = 'Mutation'
            ids = disease_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        ys_train, metrics_train, ys_test, metrics_test = run_rf(train_index_all, test_index_all, samples, 'SGD')

5it [00:00, 557.00it/s]

# Drug = 184 | Mutation = 603
# Test: Drug = 36 | Mutation = 120
========== isbalance = True | task = Tp
-------Fold  0
# Pairs: Train = 1470 | Test = 368
-------Fold  1
# Pairs: Train = 1470 | Test = 368
-------Fold  2
# Pairs: Train = 1470 | Test = 368
-------Fold  3
# Pairs: Train = 1471 | Test = 367
-------Fold  4
# Pairs: Train = 1471 | Test = 367
-----------------------Fold =  0


AttributeError: probability estimates are not available for loss='hinge'

In [68]:
directory='C:/Users/Administrator/Desktop/图采样data/last data'
#directory = 'data'
for isbalance in [True]:
#for isbalance in [True, False]:
    
    ID, IM, dtp, mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples = obtain_data(directory, 
                                                                                                 isbalance)
    for task in ['Tp', 'Tm', 'Td']:
        
        print('========== isbalance = {} | task = {}'.format(isbalance, task))
        
        if task == 'Tp':
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(samples)
            
        elif task == 'Tm':
            item = 'Drug'
            ids = mirna_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        elif task == 'Td':
            item = 'Mutation'
            ids = disease_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, dtp)

        ys_train, metrics_train, ys_test, metrics_test = run_rf(train_index_all, test_index_all, samples, 'SVM')

5it [00:00, 456.74it/s]

# Drug = 184 | Mutation = 603
# Test: Drug = 36 | Mutation = 120
========== isbalance = True | task = Tp
-------Fold  0
# Pairs: Train = 1470 | Test = 368
-------Fold  1
# Pairs: Train = 1470 | Test = 368
-------Fold  2
# Pairs: Train = 1470 | Test = 368
-------Fold  3
# Pairs: Train = 1471 | Test = 367
-------Fold  4
# Pairs: Train = 1471 | Test = 367
-----------------------Fold =  0


AttributeError: predict_proba is not available when  probability=False